In [1]:
from pathlib import Path
import os
import torch

In [5]:
p_ckpt = Path("/data/soyeonhong/nlq/nlq_lightning/outputs/batch/2024-12-04/154138/epoch=39-val_R1_03=val_R1_03=7.508.ckpt")
state_dict = torch.load(p_ckpt, map_location="cpu", weights_only=True)['state_dict']
state_dict.keys()

odict_keys(['model.v_emb', 'model.lm.shared.weight', 'model.lm.encoder.embed_tokens.weight', 'model.lm.encoder.block.0.layer.0.SelfAttention.q.weight', 'model.lm.encoder.block.0.layer.0.SelfAttention.k.weight', 'model.lm.encoder.block.0.layer.0.SelfAttention.v.weight', 'model.lm.encoder.block.0.layer.0.SelfAttention.o.weight', 'model.lm.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'model.lm.encoder.block.0.layer.0.layer_norm.weight', 'model.lm.encoder.block.0.layer.1.DenseReluDense.wi_0.weight', 'model.lm.encoder.block.0.layer.1.DenseReluDense.wi_1.weight', 'model.lm.encoder.block.0.layer.1.DenseReluDense.wo.weight', 'model.lm.encoder.block.0.layer.1.layer_norm.weight', 'model.lm.encoder.block.1.layer.0.SelfAttention.q.weight', 'model.lm.encoder.block.1.layer.0.SelfAttention.k.weight', 'model.lm.encoder.block.1.layer.0.SelfAttention.v.weight', 'model.lm.encoder.block.1.layer.0.SelfAttention.o.weight', 'model.lm.encoder.block.1.layer.0.layer_norm.weight', 'mode